# 学习的开始

笔记用于记录学习《Python绝技》这本书的过程中输出的代码、文字，因此采用jupyter notebook作为记录和分享的方式。很喜欢这个工具。

在笔记中，除了编写和运行相应功能的代码之外，同时把每个部分所使用到的工具、数据都进行说明。全部学完之后，也许会在README中把所有需要的工具和Python库做一下整理。

话不多说，下面正式进入学习！

# UNIX口令破解

Unix系统计算口令的算法是加盐（salt）的crypt，调用crypt方法，将salt和口令作为参数计算出一个hash值。Python的crypt库中带有crypt()方法可以用来计算这个hash值。

假设口令为“jupyter”，salt为“NT”，计算得到hash值为：

In [8]:
import crypt
salt = "NT"

# case 1:
passwd = "jupyterr" 
crypt.crypt(passwd, salt)

'NT/Xpm78Dk6hk'

In [7]:
# case 2:
passwd = "jupyterrere"
crypt.crypt(passwd, salt)

'NT/Xpm78Dk6hk'

通过这个简单的例子可以观察到，加密后的hash值**前两位是salt本身**。我们根据Unix的crypt函数计算方法可以知道，crypt算法基于**DES**加密算法[^DES]，DES是基于字符置换的密码算法，salt用于扰动算法。crypt算法接受一个长度不超过8的口令，从上面两个例子对比可知，如果长度超过8，后面的字符会被丢弃，因此两个计算结果是相同的。salt是长度为2的字符串。

这种情况下计算出来的密码是很难防碰撞的，因此实际上Linux内部用了更有效的加密算法[^linux_crypt]。不过我们仍然可以用crypt来理解口令破解的过程。

针对上述情况，我们可以在获得密文的前提下，使用词库中的词汇进行计算和hash匹配，碰撞得到口令明文。因此，如果使用的是弱口令，就容易被枚举出来破解。简单写个破解算法：

In [34]:
import crypt

# supposed that the thesaurus have only these words but in fact it can be read from a file and have lots of candidate passwords. 
thesaurus=["jupyter", "123456", "abcdef", "notebook"] 

def test_pwd(user_pwd):
    user = user_pwd.split(':')[0].strip()
    cipher_text = user_pwd.split(":")[1].strip()
    
    if len(cipher_text) != 13:
        raise Exception(f"length of cipher text must be 13 (found: {len(cipher_text)}; `{user}: {cipher_text}`)!")
    salt = cipher_text[:2]
        
    # enum all candidate password    
    for w in thesaurus:
        if (crypt.crypt(w, salt) == cipher_text):
            print(f"[+] plain password of `{user}: {cipher_text}` found: `{w}`")
            return
    print(f"[+] plain password of `{user}: {cipher_text}` not found")

# test three cipher text 
user_pwd = ['user1:d36j1XGIvC.6g', 'hello1:WH8PqwO5uDWMc', 'nobody:Y7/8PqwOc']
for up in user_pwd:
    try:
        test_pwd(up)
    except Exception as e:
        print(f"[-] {str(e)}")

[+] plain password of `user1: d36j1XGIvC.6g` found: `notebook`
[+] plain password of `hello1: WH8PqwO5uDWMc` not found
[-] length of cipher text must be 13 (found: 9; `nobody: Y7/8PqwOc`)!


# 参考资料

[^DES]: [DES算法](https://baike.baidu.com/item/des%E7%AE%97%E6%B3%95/10306073#:~:text=DES%E7%AE%97%E6%B3%95%E4%B8%BA%20%E5%AF%86%E7%A0%81%E4%BD%93%E5%88%B6%20%E4%B8%AD%E7%9A%84%E5%AF%B9%E7%A7%B0%E5%AF%86%E7%A0%81%E4%BD%93%E5%88%B6%EF%BC%8C%E5%8F%88%E8%A2%AB%E7%A7%B0%E4%B8%BA%E7%BE%8E%E5%9B%BD,%E6%95%B0%E6%8D%AE%E5%8A%A0%E5%AF%86%E6%A0%87%E5%87%86%20%EF%BC%8C%E6%98%AF1972%E5%B9%B4%E7%BE%8E%E5%9B%BDIBM%E5%85%AC%E5%8F%B8%E7%A0%94%E5%88%B6%E7%9A%84%E5%AF%B9%E7%A7%B0%E5%AF%86%E7%A0%81%E4%BD%93%E5%88%B6%20%E5%8A%A0%E5%AF%86%E7%AE%97%E6%B3%95%20%E3%80%82)
[^linux_crypt]: [linux crypt函数](https://blog.csdn.net/liuxingen/article/details/46673305)